In [10]:
import logging
import time
from pathlib import Path
import json 
import yaml

from docling_core.types.doc import ImageRefMode, PictureItem, TableItem

from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.transforms.chunker import HierarchicalChunker

In [11]:
_log = logging.getLogger(__name__)
IMAGE_RESOLUTION_SCALE = 2.0

In [12]:
logging.basicConfig(level=logging.INFO)

input_doc_path = Path("./paciente.pdf")
output_dir = Path("scratch")

pipeline_options = PdfPipelineOptions()
pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

conv_result = doc_converter.convert(input_doc_path)
# chunks = list(HierarchicalChunker().chunk(conv_result))

output_dir.mkdir(parents=True, exist_ok=True)
doc_filename = conv_result.input.file.stem

# Save page images
for page_no, page in conv_result.document.pages.items():
    page_no = page.page_no
    page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
    with page_image_filename.open("wb") as fp:
        page.image.pil_image.save(fp, format="PNG")

# Save images of figures and tables
table_counter = 0
picture_counter = 0
for element, _level in conv_result.document.iterate_items():
    if isinstance(element, TableItem):
        table_counter += 1
        element_image_filename = (
            output_dir / f"{doc_filename}-table-{table_counter}.png"
        )
        with element_image_filename.open("wb") as fp:
            element.get_image(conv_result.document).save(fp, "PNG")

    if isinstance(element, PictureItem):
        picture_counter += 1
        element_image_filename = (
            output_dir / f"{doc_filename}-picture-{picture_counter}.png"
        )
        with element_image_filename.open("wb") as fp:
            element.get_image(conv_result.document).save(fp, "PNG")


# Save markdown with embedded pictures
md_filename = output_dir / f"{doc_filename}-with-images.md"
conv_result.document.save_as_markdown(md_filename, image_mode=ImageRefMode.EMBEDDED)

# Save markdown with externally referenced pictures
md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
conv_result.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

# Save json
json_filename = output_dir / f"{doc_filename}.json"
conv_result.document.save_as_json(json_filename, image_mode=ImageRefMode.REFERENCED)

# Save yaml
yaml_filename = output_dir / f"{doc_filename}.yaml"
conv_result.document.save_as_yaml(yaml_filename, image_mode=ImageRefMode.REFERENCED)


INFO:docling.document_converter:Going to convert document batch...


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

INFO:docling.pipeline.base_pipeline:Processing document paciente.pdf
INFO:docling.document_converter:Finished converting document paciente.pdf in 95.40 sec.


AttributeError: 'str' object has no attribute 'save'